In [3]:
import pandas as pd
import sklearn
import tensorflow as tf

In [4]:
dataset = pd.read_csv("data/dataset_diving_behaviour.tabular", sep="\t")
dataset

,lat,lon,alt,unix,bird,species,year,date_time,max_depth.m,colony2,coverage_ratio,is_dive,is_dive_1m,is_dive_2m,is_dive_4m,is_dive_5m,is_dive_0m
0,56.095451,-6.233089,-23.059999,1340627854,1,tCOGU,t2012,2012-06-25 13:37:34,-2.172046,1,0.5,0,False,False,False,False,False
1,56.095408,-6.233520,-2.983077,1340627954,1,tCOGU,t2012,2012-06-25 13:39:14,-1.152306,1,0.6,0,False,False,False,False,False
2,56.095437,-6.234275,3.470286,1340628054,1,tCOGU,t2012,2012-06-25 13:40:54,-2.172046,1,0.7,0,False,False,False,False,False
3,56.095635,-6.234815,1.902667,1340628154,1,tCOGU,t2012,2012-06-25 13:42:34,-2.172046,1,0.8,0,False,False,False,False,False
4,56.095821,-6.235293,2.824952,1340628254,1,tCOGU,t2012,2012-06-25 13:44:14,-2.172046,1,0.9,0,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263713,57.380016,-1.876185,38.550566,1339500344,108,tCOGU,t2012,2012-06-12 12:25:44,-1.152306,10,NaN,0,False,False,False,False,False
263714,57.380147,-1.876534,32.490545,1339500444,108,tCOGU,t2012,2012-06-12 12:27:24,-1.152306,10,NaN,0,False,False,False,False,False
263715,57.380096,-1.876400,38.292069,1339500544,108,tCOGU,t2012,2012-06-12 12:29:04,1.906914,10,NaN,0,True,False,False,False,True
263716,57.380045,-1.876275,38.851776,1339500644,108,tCOGU,t2012,2012-06-12 12:30:44,-2.172046,10,NaN,0,False,False,False,False,False


In [9]:
use_data = dataset[["lat", "lon", "alt", "coverage_ratio", "is_dive"]]
use_data = use_data.dropna()
labels = use_data[["is_dive"]]

In [12]:
import numpy as np
final_features = list()

window_size = 50
break_point = 100000
tr_features = ["lat", "lon", "alt", "coverage_ratio"]

for i in range(use_data.shape[0] - window_size):
    norm_features = list()
    win_df = use_data[i:i+window_size]
    for feature in tr_features:
        mean_feature = win_df[feature].mean()
        #print(feature, mean_feature)
        norm_feature = win_df[feature] - mean_feature
        norm_features.extend(norm_feature.tolist())
        #printfinal_features(len(norm_feature.tolist()), len(norm_features), norm_feature.tolist())
        #print()
    #print(norm_features)
    norm_features.extend([int(use_data.iloc[i]["is_dive"])])
    #print(len(norm_features), norm_features[200])
    final_features.append(norm_features)
    if i == break_point:
        break

final_features = np.array(final_features)
print(final_features.shape)

(100001, 201)


In [18]:
labels = final_features[:,-1]
labels.shape

(100001,)

In [20]:
tr_features = final_features[:,:-1]
tr_features.shape

(100001, 200)

In [23]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, input_shape=(tr_features.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(tr_features, labels, batch_size=256, epochs=20, validation_split=0.2)

Epoch 1/20
313/313 [==============================] - 2s 7ms/step - loss: 1.1808 - val_loss: 0.8953
Epoch 2/20
313/313 [==============================] - 4s 13ms/step - loss: 0.5817 - val_loss: 0.7383
Epoch 3/20
313/313 [==============================] - 5s 16ms/step - loss: 0.4935 - val_loss: 0.6769
Epoch 4/20
313/313 [==============================] - 5s 17ms/step - loss: 0.4587 - val_loss: 0.6367
Epoch 5/20
313/313 [==============================] - 4s 14ms/step - loss: 0.4308 - val_loss: 0.6219
Epoch 6/20
313/313 [==============================] - 4s 12ms/step - loss: 0.4156 - val_loss: 0.5722
Epoch 7/20
313/313 [==============================] - 4s 13ms/step - loss: 0.3977 - val_loss: 0.5359
Epoch 8/20
313/313 [==============================] - 5s 15ms/step - loss: 0.3897 - val_loss: 0.5410
Epoch 9/20
313/313 [==============================] - 4s 13ms/step - loss: 0.3822 - val_loss: 0.5276
Epoch 10/20
313/313 [==============================] - 5s 15ms/step - loss: 0.3809 - val_los